# Setup ToyData

In [67]:
class ToyData:

    def __init__(self):

        self.attributes = {'color':['y', 'g', 'b'], 'size':['s','l'], 'shape':['r', 'i']}
        self.classes = ('+', '-')

        self.data = [('y', 's', 'r'),
                 ('y', 's', 'r'),
                 ('g', 's', 'i'),
                 ('g', 'l', 'i'),
                 ('y', 'l', 'r'),
                 ('y', 's', 'r'),
                 ('y', 's', 'r'),
                 ('y', 's', 'r'),
                 ('g', 's', 'r'),
                 ('y', 'l', 'r'),
                 ('y', 'l', 'r'),
                 ('y', 'l', 'r'),
                 ('y', 'l', 'r'),
                 ('y', 'l', 'r'),
                 ('y', 's', 'i'),
                 ('y', 'l', 'i')]
        self.target = ('+', '-', '+', '-', '+', '+', '+', '+', '-', '-', '+', '-', '-', '-', '+', '+')

        self.testData = [('y', 's', 'r'),
                 ('y', 's', 'r'),
                 ('g', 's', 'i'),
                 ('g', 'l', 'i'),
                 ('y', 'l', 'r')]

        self.testTarget = ('+', '-', '+', '-', '+')

    def get_data(self):
        return self.attributes, self.classes, self.data, self.target, self.testData, self.testTarget



# Implement decision tree classifier based on the ID3 algorithm

In [185]:
from collections import Counter
from graphviz import Digraph


In [223]:
class ID3DecisionTreeClassifier :
    def __init__(self, minSamplesLeaf = 1, minSamplesSplit = 2) :

        self.__nodeCounter = 0

        # the graph to visualise the tree
        self.__dot = Digraph(comment='The Decision Tree')

        # suggested attributes of the classifier to handle training parameters
        self.__minSamplesLeaf = minSamplesLeaf
        self.__minSamplesSplit = minSamplesSplit


    # Create a new node in the tree with the suggested attributes for the visualisation.
    # It can later be added to the graph with the respective function
    def new_ID3_node(self):
        node = {'id': self.__nodeCounter, 'label': None, 'attribute': None, 'entropy': None, 'samples': None,
                         'classCounts': None, 'nodes': None}

        self.__nodeCounter += 1
        return node

    # adds the node into the graph for visualisation (creates a dot-node)
    def add_node_to_graph(self, node, parentid=-1):
        nodeString = ''
        for k in node:
            if ((node[k] != None) and (k != 'nodes')):
                nodeString += "\n" + str(k) + ": " + str(node[k])

        self.__dot.node(str(node['id']), label=nodeString)
        if (parentid != -1):
            self.__dot.edge(str(parentid), str(node['id']))

    # make the visualisation available
    def make_dot_data(self) :
        return self.__dot

    # For you to fill in; Suggested function to find the best attribute to split with, given the set of
    # remaining attributes, the currently evaluated data and target.
    def find_split_attr(self, data,target,attributes,classes):
        max_info_gain = None
        for attr in attributes:
            remaining_attr = self.removekey(attributes,attr)
            i_gain, ent = self.info_gain(attr, data,target,attributes,classes)
            print(ent)
            if max_info_gain is None or i_gain > max_info_gain:
                max_info_gain = i_gain
                max_info_gain_attr = attr
                
        return max_info_gain_attr, ent
    
    def removekey(self, d, key):
        r = dict(d)
        del r[key]
        return r
    
    def entropy_split(self,data,col,val,counter):
        
        for i in range(len(data)):
            if data[i][col] == val:
                cnt[target[i]] += 1
            
        n_v = len(list(cnt.elements()))
        ent_v = 0
        for cl in classes:
            p_x = cnt[cl]/n_v
            ent_v += -p_x*math.log(p_x,2)
        return ent_v, n_v
    
    def info_gain(self,split, data,target,attributes,classes):
        # Calculate the entropy first
        entropy = 0
        cnt = Counter(target)
        n = len(target)
        
        for cl in classes:
            p_x = cnt[cl]/n
            entropy += - p_x * math.log(p_x)
        
        col = list(attributes.keys()).index(split)
        info_gain = entropy
        for val in attributes[split]:
            cnt = Counter()
            ent_v, n_v = self.entropy_split(data, col, val, cnt)
            info_gain += - n_v/n*ent_v
            
        return info_gain, entropy
            
                
    # Use this function split the data acording to the split attribute
    # Return values are dicts with value of split attribute as keys and the data/targets as items.
    def find_data_split(self,data,target,attributes,classes,split):
        data_split = {}
        target_split = {}
        if len(attributes) > 1:
            col = list(attributes.keys()).index(split)
        else:
            col = 0
        for vals in attributes[split]:
            data_split[vals] = []
            target_split[vals] = []
            for i in range(len(target)):
                if data[i][col] == vals:
                    data_split[vals].append(data[i])
                    target_split[vals].append(target[i])
                   
                    
        return data_split, target_split
        

    # the entry point for the recursive ID3-algorithm, you need to fill in the calls to your recursive implementation
    def fit(self, data, target, attributes, classes):
        node = self.new_ID3_node()
        self.add_node_to_graph(node)
        cnt = Counter(target)
        node['samples'] = len(target)
        node['classCounts'] = cnt
        if len(cnt) == 1:
            node['label'] = target[0]
            return node
        
        elif len(attributes) == 0:
            node['label'] = cnt.most_common(1)[0][0]
            return node
        else:
            split,ent = self.find_split_attr(data,target,attributes,classes)
            node['nodes'] = {}
            node['entropy'] = ent
            node['attribute'] = split
            remaining_attr = self.removekey(attributes, split)
            fit_data, fit_target = self.find_data_split(data,target,attributes,classes,split)    
            
            for vals in attributes[split]:
                if len(fit_target[vals]) == 0:
                    leaf_node = self.new_ID3_node()
                    leaf_node['label'] = cnt.most_common(1)[0][0]
                    node['nodes'][vals] = leaf_node
                else:
                    node['nodes'][vals] = self.fit(fit_data[vals],fit_target[vals],remaining_attr,classes)

        # fill in something more sensible here..  root should become the output of the recursive tree creation
        # root = self.new_ID3_node()
        # self.add_node_to_graph(root)

        return node
    

    def predict(self, data, tree) :
        predicted = list()

        # fill in something more sensible here... root should become the output of the recursive tree creation
        return predicted

# Run the ID3 classifier

In [224]:
#import ToyData as td
#import ID3

import numpy as np
import math
from sklearn import tree, metrics, datasets


In [225]:
attributes, classes, data, target, data2, target2 = ToyData().get_data()

id3 = ID3DecisionTreeClassifier()

myTree = id3.fit(data, target, attributes, classes)
print(myTree)
plot = id3.make_dot_data()
plot.render("testTree")
# predicted = id3.predict(data2, myTree)
# print(predicted)

0.6853142072764582
0.6853142072764582
0.6853142072764582
0.5623351446188083
0.5623351446188083
0.6615632381579821
0.6615632381579821
{'id': 0, 'label': None, 'attribute': 'size', 'entropy': 0.6853142072764582, 'samples': 16, 'classCounts': Counter({'+': 9, '-': 7}), 'nodes': {'s': {'id': 1, 'label': None, 'attribute': 'shape', 'entropy': 0.5623351446188083, 'samples': 8, 'classCounts': Counter({'+': 6, '-': 2}), 'nodes': {'r': {'id': 2, 'label': '+', 'attribute': None, 'entropy': None, 'samples': None, 'classCounts': None, 'nodes': None}, 'i': {'id': 3, 'label': '+', 'attribute': None, 'entropy': None, 'samples': None, 'classCounts': None, 'nodes': None}}}, 'l': {'id': 4, 'label': None, 'attribute': 'shape', 'entropy': 0.6615632381579821, 'samples': 8, 'classCounts': Counter({'-': 5, '+': 3}), 'nodes': {'r': {'id': 5, 'label': '-', 'attribute': None, 'entropy': None, 'samples': None, 'classCounts': None, 'nodes': None}, 'i': {'id': 6, 'label': '-', 'attribute': None, 'entropy': None, '

'testTree.pdf'

In [213]:
attributes
split = 'color'
col = list(attributes.keys()).index(split)

In [111]:
split = 'color'
col = list(attributes.keys()).index(split)

def entropy_split(data,col,val,counter):
    n_v = 0
    ent_v = 0
    for i in range(len(data)):
        if data[i][col] == val:
            cnt[target[i]] += 1
            n_v += 1
        
    for cl in classes:
        p_x = cnt[cl]/n_v
        ent_v += -p_x*math.log(p_x,2)
    return ent_v, n_v

n = len(data)
entropy = 0
info_gain = entropy
for val in attributes[split]:
    cnt = Counter()
    ent_v, n_v = entropy_split(data, col, val, cnt)
    info_gain += - n_v/n*ent_v
            
            
            

ZeroDivisionError: division by zero

In [226]:
attributes, classes, data, target, data2, target2 = ToyData().get_data()

In [234]:
cnt = Counter(target)
cnt.most_common(1)[0][0]


'+'

'+'